# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [4]:
import re
from collections import Counter
from queue import Queue
import nltk
import math
import heapq
import random

In [5]:
class SpellChecker:
    def __init__(self, use_keyboard=True, n_range=[2, 5], search_breadths=10):
        """
        Initialization with all the preferences and keyboard matrix
        """
        self.name = f"Spell checker n-gram based model{' with keyboard mapping' if use_keyboard else ''}"
        self.use_keyboard = use_keyboard
        self.keyboard_matrix = {'q': ['w', 'a'], 'w': ['q', 'a', 's', 'e'], 'e': ['w', 's', 'd', 'r'],
                                'r': ['e', 'd', 'f', 't'], 't': ['r', 'f', 'g', 'y'], 'y': ['t', 'g', 'h', 'u'],
                                'u': ['y', 'h', 'j', 'i'], 'i': ['u', 'j', 'k', 'o'], 'o': ['i', 'k', 'l', 'p'],
                                'p': ['o', 'l'], 'a': ['q', 'w', 's', 'z'], 's': ['a', 'w', 'e', 'd', 'z', 'x'],
                                'd': ['s', 'e', 'r', 'f', 'c', 'x'], 'f': ['d', 'r', 't', 'g', 'c', 'v'],
                                'g': ['f', 't', 'y', 'h', 'v', 'b'], 'h': ['g', 'y', 'u', 'b', 'n', 'j'],
                                'j': ['h', 'u', 'i', 'k', 'n', 'm'], 'k': ['j', 'i', 'l', 'm'], 'l': ['k', 'o', 'p'],
                                'z': ['a', 's', 'x'], 'x': ['z', 's', 'd', 'c'], 'c': ['x', 'd', 'f', 'v'],
                                'v': ['c', 'f', 'g', 'b'], 'b': ['v', 'g', 'h', 'n'], 'n': ['b', 'h', 'j', 'm'],
                                'm': ['n', 'j', 'k']}
        self.ngram_vocab = Counter()
        self.ngram_total_by_len = dict()
        self.words = Counter()
        self.total_words = 0
        self.top_k = search_breadths
        self.n_range = n_range
        for n in range(n_range[0], n_range[1] + 1):
            self.ngram_total_by_len[n] = 0

    def load_ngrams(self, filename):
        """
        Method to load the dataset of ngrams and fill the ngram_vocab with all possible n_range-grams
        """
        ngrams = []
        with open(filename) as ngram_list:
            ngrams = ngram_list.readlines()
        for i in range(len(ngrams)):
            txt = ngrams[i]
            count, ngram = int(txt.split()[0]), txt.split()[1:]
            for w in ngram:
                self.words[w] += count
                self.total_words += count
            for _n in range(self.n_range[0], self.n_range[1] + 1):
                window = [0, min(len(ngram), _n)]
                while window[1] < len(ngram):
                    self.ngram_vocab[' '.join(ngram[window[0]:window[1]])] += count
                    self.ngram_total_by_len[_n] += count
                    window[0] += 1
                    window[1] += 1
                if len(ngram) == _n:
                    break

    def ngram_score(self, ngram):
        """
        Method to calculate total ngram score. This score is used to rank the candidate sentences after the correction.
        Ngram score is calculated by the formula: n^2 * (sum of frequencies (in the loaded ngrams data) of all n-grams found in the sentence)
        for all integer n in n_range
        """
        if not ngram:
            return 0
        wordset = ngram.split() if type(ngram) == str else ngram
        score = 0
        subsets = [' '.join(wordset)]
        n = len(wordset)
        while n >= self.n_range[0]:
            subsets = []
            n -= 1
            window = [0, n]
            while window[1] < len(wordset):
                subsets.append(' '.join(wordset[window[0]:window[1]]))
                window[0] += 1
                window[1] += 1
            score += max(1, n ** 2) * sum([self.ngram_vocab[subset] / self.ngram_total_by_len[n] if subset in
                                          self.ngram_vocab else 0 for subset in subsets])
        return score

    def word_prob(self, word):
        """
        Method to get a word probability (frequency in the loaded dataset), same as P calculation in Norwig's solution
        """
        return self.words[word] / self.total_words

    def known(self, words):
        return set(w for w in words if w in self.words)

    def candidates(self, word):
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def edits1(self, word):
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def correct_word(self, word):
        results = self.known([word])

        if len(results) == 0:
            results = results.union(self.known(self.edits1(word)))
            results = results.union(self.known(self.edits2(word)))

        if len(results) > 0:
            edit_distances = dict()
            for corrected_word in results:
                dist = self.edit_distance(corrected_word, word)
                edit_distances[dist] = edit_distances.get(dist, [])
                edit_distances[dist].append(corrected_word)
            sorted_dist = sorted(list(edit_distances.keys()))
            score = []
            for d, ws in edit_distances.items():
                score.extend((w, self.words[w] * 1e5 / (self.total_words * max(1, d ** 4))) for w in ws)
            top_k_options = heapq.nlargest(self.top_k, score, key=lambda x: x[1])
            top_k_words = [option[0] for option in top_k_options]
            return top_k_words
        else:
            return [word]

    def correct_sentence(self, sentence):
        sentence = list(map(lambda x: x.lower(), sentence.split() if type(sentence) == str else sentence))
        options = []
        corrected_sentences = self.generate_corrected_sentences(sentence)
        for corrected_sentence in corrected_sentences:
            if len(corrected_sentence) == len(sentence):
                total_score = self.ngram_score(corrected_sentence)
                options.append((total_score, corrected_sentence))
        options.sort(key=lambda x: x[0], reverse=True)
        return ' '.join(options[0][1])

    def generate_corrected_sentences(self, words):
        if not words:
            return [[]]

        rest_sentences = self.generate_corrected_sentences(words[1:])
        first_word_sentences = self.correct_word(words[0])
        corrected_sentences = []
        for first_word in first_word_sentences:
            for rest_sentence in rest_sentences:
                corrected_sentences.append([first_word] + rest_sentence)
        return corrected_sentences

    def edit_distance(self, corrected_word, original_word):
        """
        Method to calculate the edit distance between the original and corrected words.
        Reuses code from the Spell_checker by Almaz Samatov, provided in the explanation section
        """
        def get_distance_between_letters(letter1, letter2):
            """
            Nested function for calculation of distance between the given letters.
            Reuses code from the Spell_checker by Almaz Samatov, provided in the explanation section
            """
            q = Queue()
            q.put(letter1)
            used = set()
            used.add(letter1)
            dist = dict()
            dist[letter1] = 0
            while not q.empty():
                symbol = q.get()
                for symbol_to_go in self.keyboard_matrix[symbol]:
                    if symbol_to_go not in used:
                        used.add(symbol_to_go)
                        q.put(symbol_to_go)
                        dist[symbol_to_go] = dist[symbol] + 1
            if dist[letter2] <= 2:
                return dist[letter2]
            else:
                return 3

        distance = [[0 for i in range(len(corrected_word) + 1)] for j in range(len(original_word) + 1)]

        for i in range(len(original_word) + 1):
            for j in range(len(corrected_word) + 1):
                if min(i, j) == 0:
                    distance[i][j] = max(i, j)
                else:
                    deletion_cost = distance[i - 1][j] + 2

                    if corrected_word[j - 1] == original_word[i - 1]:
                        insertion_cost = distance[i][j - 1] + 1
                    else:
                        insertion_cost = distance[i][j - 1] + 3

                    substitution_cost = distance[i - 1][j - 1] + get_distance_between_letters(original_word[i - 1],
                                                                                              corrected_word[j - 1])

                    if i > 1 and j > 1 and corrected_word[j - 1] == original_word[i - 2] \
                            and corrected_word[j - 2] == original_word[i - 1]:
                        transposition_cost = distance[i - 2][j - 2] + 1
                        distance[i][j] = min(deletion_cost, insertion_cost, substitution_cost, transposition_cost)
                    else:
                        distance[i][j] = min(deletion_cost, insertion_cost, substitution_cost)

        return distance[len(original_word)][len(corrected_word)]


    def deform_word(self, word, distance_probs={0: 0.4, 1: 0.3, 2: 0.2, 3: 0.05, 4: 0.05}):
        """
        Method that reuses the methods from the above to deform the word
        accordingly to the distance probabilities given.
        Uses some aproaches to lower the time needed to deform one word with a good quality
        """
        num = random.random()
        if 0 in distance_probs and num <= distance_probs[0]:
            return word
        ed1s = list(self.edits1(word))
        random.shuffle(ed1s)
        deformed = [word] + ed1s[:min(len(ed1s), max(10, int(0.025 * len(ed1s))))]

        guaranteed_one_edit_prob = distance_probs[0] if 0 in distance_probs else 0 + \
                                   distance_probs[1] if 1 in distance_probs else 0
        if num > guaranteed_one_edit_prob:
            ed2s = list(self.edits2(word))
            random.shuffle(ed2s)
            deformed.extend(ed2s[:min(len(ed2s), 10)])

        edit_distances = dict()
        for w in deformed:
            dist = self.edit_distance(w, word)
            edit_distances[dist] = edit_distances.get(dist, [])
            edit_distances[dist].append(w)
        sorted_dist = sorted(list(edit_distances.keys()))

        candidates = []
        for item in distance_probs.items():
            pass

        cumulative_probs = [0]
        for dist, prob in distance_probs.items():
            cumulative_probs.append(cumulative_probs[-1] + prob)

        for i in range(len(cumulative_probs) - 1):
            if cumulative_probs[i] <= num < cumulative_probs[i + 1]:
                if len(sorted_dist) <= i:
                    i = len(sorted_dist) - 1
                target_dist = sorted_dist[i]
                break

        if target_dist in edit_distances:
            return random.choice(edit_distances[target_dist])
        else:
            return word

In [6]:
spc = SpellChecker()  # My spellchecker is easy to use. Just init, load_ngrams and then you are ready to go!
spc.load_ngrams('fivegrams.txt')  # I've loaded fivegrams to allow usage of all ngrams in n_range=[2, 5]

In [7]:
'''
Some examples to figure out the possibilities
'''
print('Bib af reah hollwo  --> ', spc.correct_sentence('Bib af reah hollwo'))
print('Thib is an exampel sntrnce  --> ', spc.correct_sentence('Thib is an exampel sntrnce'))
print('tell me eqy I fillow youu  --> ', spc.correct_sentence('tell me eqy I fillow youu'))
print('a bbleu sea is far awqy frpm me  --> ', spc.correct_sentence('a bbleu sea is far awqy frpm me'))

Bib af reah hollwo  -->  it was great hollow
Thib is an exampel sntrnce  -->  this is an example sentence
tell me eqy I fillow youu  -->  tell me way i follow you
a bbleu sea is far awqy frpm me  -->  a blue sea is far away from me


In [8]:
'''
Here is the vanilla code of Norwig's solution for sentence correction
'''


def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('fivegrams.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norwig_correct_sentence(sentence):
    words_involved = list(map(lambda x: x.lower(), sentence.split()))
    for i, w in enumerate(words_involved):
        words_involved[i] = correction(w)
    return ' '.join(words_involved)

In [9]:
print(norwig_correct_sentence('Bib af reah hollwo'))
print(norwig_correct_sentence('Thib is an exampel sntrnce'))
print(norwig_correct_sentence('tell me eqy I fillow youu'))
print(norwig_correct_sentence('a bbleu sea is far awqy frpm me'))

big of real hollow
this is an example entrance
tell me way i follow you
a able sea is far away from me


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

For more context, I have used 5grams dataset including all the (2, 5)grams (from bigrams to fivegrams). One may change the hyperparameter (even to larger n if one is able to find dataset for ngrams) to consider other gram sizes

Weights are assigned accordingly to the Damerau-Levenshtein edit distance between corrected and original word, taking into account the neighboring words on the keyboard.
For example, word 'now' will have less edit distance from the misspelled word 'noq' than the word 'not'.

All the weights are applied on different stages of candidates preparation or the exact candidate choice:


*   edit distance is affecting the stage of word correction, which is made before the candidate sentences choice (candidate sentences are all sentences with correct words which can be constructed by 0-2 edits of the initial ones, even if the initial words are present in the vocabulary). Note that two misclicked letters which are close on the keyboard to the corrected ones can give lower distance than one misclick from far away.
*   word probability P is applied at the same place where edit distance is applied (so the weight is the following: 1e5 * P/(edit_distance**4))
*   ngram probability is calculated during the final choice among the candidate sentences. It takes into account all the (2, 5)grams (or, again, another predefined by a parameter gram set), and the more words in the ngram that is present in the sentence, the bigger score it will get and will have bigger probability to be chosen

Beam search here is altered by the techniques described above (not a direct vanilla beam-search, but candidate filtering).


The ideas, implementation parts, concept mappings, namings, etc. were taken from different sources to present a unique self-implemented solution. As references, I would like to put the following:

[Norwig's solution](https://norvig.com/spell-correct.html) that is a baseline was also used as a base code for the above solution;

[Spelling correction guide](https://towardsdatascience.com/spelling-correction-how-to-make-an-accurate-and-fast-corrector-dc6d0bcbba5f) by Filipp Bakanov with awesome ideas how to iteratively improve the Norwig's solution, with quantitative and qualitative analysis. Here I've taken the idea of how to use ngrams. One can find here further improvements and next steps;

[Spell_checker](https://github.com/AlmazSamatov/Spell_checker/tree/master) by Almaz Samatov, a solution to introduce keyboard graph and edit distance over the Norwig's solution. I've taken the edit distance and keyboard mapping dictionary from it to avoid implementing it from scratch;

[SymSpell](https://wolfgarbe.medium.com/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f) have inspired me on how to reduce the number of computations (but I am far away from 1000x faster correction presented there)

[Yandex Cup on ML](https://yandex.ru/cup/ml/) with its task of NeuroSwipe on the keyboard, which I was solving in 2023, helped me to outline significant points of how to weight up the distance calculated and the word probability in the dataset


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

Noise probability is used in the deform_word method in another form: a probability of edit_distance between the word and deformed one

In [18]:
import random
import nltk
from tqdm.notebook import tqdm
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


with open('big.txt') as big:
    big_s = big.readlines()

text = ' '.join(big_s)
sentences = sent_tokenize(text)
random.shuffle(sentences)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
w = sentences[0].split()[0]
print(w.lower(), spc.deform_word(w.lower(), {0: 0.1, 1: 0.2, 2: 0.3, 3: 0.4}))

muttering muytering


In [13]:
TEST_SAMPLES = 1000
norwig_total_w = 0
spc_total_w = 0
total_words = 0
norwig_total_s = 0
spc_total_s = 0


for s in tqdm(sentences[:TEST_SAMPLES]):
    orig_words = []
    sent_deformation = []
    for w in s.split():
        w = w.lower()
        temp_w = ''
        for c in w:
            if ord('a') <= ord(c) <= ord('z'):
                temp_w += c
        pref_symb, suf_symb = '', ''
        if ord('a') <= ord(w[0]) <= ord('z'):
            pref_symb = w[0]
        if ord('a') <= ord(w[-1]) <= ord('z'):
            suf_symb = w[-1]
        w = temp_w
        orig_words.append(w)
        new_w = spc.deform_word(w)
        sent_deformation.append(new_w)
        total_words += 1
    deformed_sentence = ' '.join(sent_deformation)
    norwigs_res = norwig_correct_sentence(deformed_sentence)
    spc_res = spc.correct_sentence(deformed_sentence)

    for i, w in enumerate(norwigs_res.split()):
        if w == orig_words[i]:
            norwig_total_w += 1
    if ' '.join(orig_words) == norwigs_res:
        norwig_total_s += 1

    for i, w in enumerate(spc_res.split()):
        if w == orig_words[i]:
            spc_total_w += 1
    if ' '.join(orig_words) == spc_res:
        spc_total_s += 1

  0%|          | 0/1000 [00:00<?, ?it/s]

In [17]:
print(f'Per-word accuracies: \nNorwig: {norwig_total_w / total_words} \nSpellChecker: {spc_total_w / total_words}')
print(f'Per-sentence accuracies: \nNorwig: {norwig_total_s / TEST_SAMPLES}\nSpellChecker: {spc_total_s / TEST_SAMPLES}')

Per-word accuracies: 
Norwig: 0.7813664596273292 
SpellChecker: 0.8248447204968944
Per-sentence accuracies: 
Norwig: 0.608
SpellChecker: 0.61


### For further work
I'd like to consider the improvements from Filipp Bakanov's guide, and also reconsider weights and score calculations to improve the performance. But overall, it has a good enough accuracy!

Thanks for the assignment!